In [ ]:
import os
import re
import copy
import pickle

import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import block_reduce
from skimage.transform import resize

import importlib
from IPython.display import clear_output

import data_utils
import CGAN

## Load and preprocess the data

In [ ]:
with open("./data/scaler.pkl", "rb") as f:
    scaler = pickle.load(f)
with open("./data/table.pkl", "rb") as f:
    lookup_table = pickle.load(f)
face_data = np.load("./data/face_data.npy")
landmarks = np.load("./data/landmarks.npy")

# Visualize the data
data_utils.visualize_z(face_data, z_channel=2)

## Train the model

In [ ]:
importlib.reload(data_utils);
importlib.reload(CGAN);

In [ ]:
class Params:

    drop_rate = 0.4
    n_filters = 32
    n_rand = 100
    batch_size = 64
    train_steps = 500
    interval = 1
    lr_D =  0.0001
    lr_GD = 0.0002
    
    def __init__(self, X, X_cond=None):
        _, self.W, self.H, self.n_channels = X.shape
        _, self.n_cond = X_cond.shape if X_cond else (None, 0)

In [ ]:
X = face_data[:, :, :, 2:3]
network = CGAN.CGAN(Params(X))

In [ ]:
# CHANGE SAVE SUFFIX MECHANISM
def train(network, X, train_steps, prev_steps=0):
    for step in range(train_steps):
        X_real = X[np.random.randint(X.shape[0], size=(Params.batch_size)), :, : :]
        X_fake = network.G.predict(np.random.uniform(size=(Params.batch_size, Params.n_rand)))
        X_D = np.concatenate((X_real, X_fake))
        Y_D = np.concatenate((np.ones((X_real.shape[0], 1)), np.zeros((X_fake.shape[0], 1))))
        loss_D, acc_D = network.D.train_on_batch(X_D, Y_D)

        for i in range(2):
            X_GD = np.random.uniform(size=(Params.batch_size, Params.n_rand))
            Y_GD = np.ones((X_GD.shape[0], 1))
            loss_GD, acc_GD = network.GD.train_on_batch(X_GD, Y_GD)

        if step % Params.interval == 0 and step > 0:
            print("Step {}:".format(step + prev_steps))
            print()
            data_utils.visualize_z(X_fake)
            print()
            print("Descriminator :: loss = {}, acc = {}".format(loss_D, acc_D))
            print("Adversarial   :: loss = {}, acc = {}".format(loss_GD, acc_GD))

        if step % 5 * Params.interval == 0 and step > 0:
            clear_output(wait=True)
    with open("./models/CGAN-{}-1D2G.pkl".format(prev_steps + train_steps), "wb") as f:
        pickle.dump(network, f)

In [ ]:
for i in range(5):
    train(network, X, 200, prev_steps=3000 + i * 200)

## Things to try

* Penalize overconfidence by labeling real images with 0.9

* Switch to Adam optimizer

* Switch from max-pool and upsampling to convolution stride

* Add conditional data to random noise (consider label as one hot) to account for different topology of the data, impede "averaging" of faces w.r.t. facial expressions.

* Remove dropout from generator

* Make discriminator more complex (harder to learn, smarter)

In [ ]:
for fname in sorted(os.listdir("./models/")):
    if fname.endswith(".pkl"):
        with open(os.path.join("./models/", fname), "rb") as f:
            model = pickle.load(f)
            X_fake = model.G.predict(np.random.uniform(size=(model.params.batch_size, model.params.n_rand)))
            print("Model:", fname)
            data_utils.visualize_z(X_fake)
            del model